In [1]:
# !hdfs dfs -put data/sample_movie_ratings.txt /sample_movie_ratings.txt

In [2]:
# Code from https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

lines = spark.read.option("header", "true").csv("data/interactions_train.csv").rdd
ratingsRDD = lines.map(lambda p: Row(userId=int(p[0]), recipeId=int(p[1]),
                                     rating=float(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

In [3]:
(unnorm_training, unnorm_test) = ratings.randomSplit([0.8, 0.2])
mean = unnorm_training.agg({'rating': 'mean'}).collect()[0][0]
std = unnorm_training.agg({'rating': 'std'}).collect()[0][0]
print(mean, std)
training = unnorm_training.withColumn("rating",(col("rating") - mean) / std)
test = unnorm_test.withColumn("rating",(col("rating") - mean) / std)

4.574791656605009 0.958166372324364


In [4]:
best_model = None
best_rmse = 100
for alpha in [0.01, 0.05, 0.1, 0.5, 1, 2]:
    for reg in [0.05, 0.1]:
        # Build the recommendation model using ALS on the training data
        # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
        als = ALS(rank=200, alpha=alpha, maxIter=10, regParam=reg, userCol="userId", itemCol="recipeId", ratingCol="rating",
                  coldStartStrategy="drop")
        model = als.fit(training)

        # Evaluate the model by computing the RMSE on the test data
        normalized_predictions = model.transform(test)
        predictions = normalized_predictions.withColumn(
            "rating",col("rating") * std + mean
        ).withColumn(
            "prediction",col("prediction") * std + mean
        )
        evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
                                        predictionCol="prediction")

        rmse = evaluator.evaluate(predictions)
        print(alpha, reg, rmse)
        if rmse < best_rmse:
            #print(f"Best model -> rank:{rank}, reg:{reg}, rmse:{rmse}")
            predictions.show()
            best_model = model
            best_rmse = rmse

        # Generate top 10 movie recommendations for each user
        #userRecs = model.recommendForAllUsers(10)
        # Generate top 10 user recommendations for each movie
        #recipeRecs = model.recommendForAllItems(10)

1 0.01 1.48371886416645
+------+--------+------+-------------------+
|userId|recipeId|rating|         prediction|
+------+--------+------+-------------------+
|  3794|    7508|   4.0| 10.241341926494322|
|  3794|   32045|   5.0|  4.067638789116292|
|  6357|     229|   4.0| 3.9965843885055836|
|  6357|    3722|   4.0|   4.42073889735041|
|  6357|    4174|   4.0| 4.7177544553732735|
|  6357|    5287|   5.0|  4.733217532082325|
|  6357|    8961|   0.0|-0.8306088962271225|
|  6357|    9233|   5.0|  6.697984928999947|
|  6357|   10392|   4.0|  4.731662295080575|
|  6357|   10455|   5.0| 5.7782395226640695|
|  6357|   11291|   4.0|    4.6651057272019|
|  6357|   11455|   5.0|  5.382089574138453|
|  6357|   11580|   5.0| 3.4035774470043627|
|  6357|   11642|   5.0|  4.670250358169597|
|  6357|   11644|   5.0|  4.497805618877775|
|  6357|   11749|   5.0|   4.52909787271883|
|  6357|   13366|   2.0| 3.7269355019082004|
|  6357|   13868|   4.0|  4.849276431205299|
|  6357|   14248|   5.0|  3.305

Py4JJavaError: An error occurred while calling o3444.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4351.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4351.0 (TID 8955) (aidans-air.lan executor driver): java.lang.OutOfMemoryError: GC overhead limit exceeded
	at org.apache.spark.ml.recommendation.ALS$CholeskySolver.solve(ALS.scala:792)
	at org.apache.spark.ml.recommendation.ALS$.$anonfun$computeFactors$4(ALS.scala:1766)
	at org.apache.spark.ml.recommendation.ALS$$$Lambda$3818/281597203.apply(Unknown Source)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$mapValues$3(PairRDDFunctions.scala:751)
	at org.apache.spark.rdd.PairRDDFunctions$$Lambda$3640/2119548997.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at org.apache.spark.util.collection.ExternalAppendOnlyMap.insertAll(ExternalAppendOnlyMap.scala:156)
	at org.apache.spark

In [ ]:
best_model.rank

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 53607)
Traceback (most recent call last):
  File "/Users/aidankeogh/opt/anaconda3/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/aidankeogh/opt/anaconda3/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/aidankeogh/opt/anaconda3/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/aidankeogh/opt/anaconda3/lib/python3.8/socketserver.py", line 720, in __init__
    self.handle()
  File "/Users/aidankeogh/opt/anaconda3/lib/python3.8/site-packages/pyspark/accumulators.py", line 262, in handle
    poll(accum_updates)
  File "/Users/aidankeogh/opt/anaconda3/lib/python3.8/site-packages/pyspark/accumulators.py", line 235, in poll
    if func():
  Fi